In [1]:
import os
import sys
import json
from tqdm import tqdm_notebook as tqdm
from pprint import pprint

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [2]:
BASE_DIR = '../mnt/inputs/origin'
TRN_BASE_DIR = '../mnt/inputs/features/train'
TST_BASE_DIR = '../mnt/inputs/features/test'
# BASE_DIR = '../mnt/inputs/features/test'

In [3]:
trn_df = pd.read_pickle(f'{BASE_DIR}/train.pkl.gz')
trn_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [4]:
tst_df = pd.read_csv(f'{BASE_DIR}/test.csv')
tst_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [5]:
label_df = pd.read_csv('../mnt/inputs/origin/train_labels.csv')
label_df.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [6]:
tst_df[tst_df.event_data.str.contains('false')].type.value_counts()

Activity      51699
Game          49852
Assessment     7917
Name: type, dtype: int64

In [7]:
tst_df[tst_df.event_data.str.contains('false')].type.value_counts()

Activity      51699
Game          49852
Assessment     7917
Name: type, dtype: int64

## debug targetFeatures

In [8]:
import sys
sys.path.append('../scripts')
from features.f101_target_features import targetFeatures

In [9]:
feat_params = {
    "datatype": 'test',
    "debug": True,
    "is_overwrite": True,
}

feat = targetFeatures(feat_params)

tst_labels_df = feat.calc_feature(tst_df)
tst_labels_df.head()

(4492, 13)


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    4.8s
[Parallel(n_jobs=32)]: Done  98 tasks      | elapsed:    5.3s
[Parallel(n_jobs=32)]: Done 224 tasks      | elapsed:    5.7s
[Parallel(n_jobs=32)]: Done 386 tasks      | elapsed:    6.4s
[Parallel(n_jobs=32)]: Done 557 out of 557 | elapsed:    6.8s finished


save feats to mnt/inputs/features/test/targetFeatures.pkl
writing total_bytes=115718...
calculate done!


,installation_id,game_session,num_correct,num_incorrect,accuracy_group
0,00abaee7,8b38fc0d2fd315dc,1.0,0.0,3
0,01242218,009c890ce6c4f3e3,1.0,1.0,2
1,01242218,31423dbcd717919e,1.0,1.0,2
2,01242218,597a8839a5a3468d,1.0,2.0,1
3,01242218,ab61cae5e3215355,0.0,3.0,0


In [10]:
feat_params = {
    "datatype": 'train',
    "debug": True,
    "is_overwrite": True,
}

feat = targetFeatures(feat_params)

trn_labels_df = feat.calc_feature(trn_df)
trn_labels_df.head()

(41549, 13)


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=32)]: Done 132 tasks      | elapsed:    0.7s
[Parallel(n_jobs=32)]: Done 384 tasks      | elapsed:    1.4s
[Parallel(n_jobs=32)]: Done 708 tasks      | elapsed:    2.3s
[Parallel(n_jobs=32)]: Done 1104 tasks      | elapsed:    3.4s
[Parallel(n_jobs=32)]: Done 1572 tasks      | elapsed:    4.7s
[Parallel(n_jobs=32)]: Done 2112 tasks      | elapsed:    6.3s
[Parallel(n_jobs=32)]: Done 2724 tasks      | elapsed:    8.1s
[Parallel(n_jobs=32)]: Done 3408 tasks      | elapsed:   10.1s
[Parallel(n_jobs=32)]: Done 3614 out of 3614 | elapsed:   10.6s finished


save feats to mnt/inputs/features/train/targetFeatures.pkl
writing total_bytes=1011073...
calculate done!


,installation_id,game_session,num_correct,num_incorrect,accuracy_group
0,0006a69f,6bdf9623adc94d89,1.0,0.0,3
1,0006a69f,77b8ee947eb84b4e,0.0,11.0,0
2,0006a69f,901acc108f55a5a1,1.0,0.0,3
3,0006a69f,9501794defd84e4d,1.0,1.0,2
4,0006a69f,a9ef3ecb3d1acc6a,1.0,0.0,3


In [11]:
(trn_labels_df.accuracy_group.values == label_df.accuracy_group.values).mean()

1.0

In [12]:
trn_labels_df.shape, label_df.shape

((17690, 5), (17690, 7))

In [13]:
trn_labels_df.accuracy_group.value_counts()

3    8845
0    4229
1    2411
2    2205
Name: accuracy_group, dtype: int64

In [14]:
trn_labels_df.num_correct.value_counts()

1.0    13461
0.0     4229
Name: num_correct, dtype: int64

In [15]:
trn_labels_df.num_incorrect.value_counts()

0.0     8845
1.0     3424
2.0     1739
3.0     1159
4.0      749
5.0      539
6.0      317
7.0      239
8.0      163
9.0      103
10.0      88
11.0      71
12.0      45
13.0      41
14.0      34
15.0      24
17.0      17
18.0      15
16.0      12
20.0      10
21.0       9
19.0       8
23.0       7
22.0       4
34.0       3
24.0       3
29.0       2
39.0       2
28.0       1
27.0       1
79.0       1
35.0       1
46.0       1
33.0       1
49.0       1
40.0       1
50.0       1
41.0       1
26.0       1
54.0       1
85.0       1
25.0       1
36.0       1
60.0       1
45.0       1
32.0       1
Name: num_incorrect, dtype: int64

In [16]:
pd.concat([label_df, tst_labels_df], axis=0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,accuracy,accuracy_group,game_session,installation_id,num_correct,num_incorrect,title
0,1.0,3,6bdf9623adc94d89,0006a69f,1.0,0.0,Mushroom Sorter (Assessment)
1,0.0,0,77b8ee947eb84b4e,0006a69f,0.0,11.0,Bird Measurer (Assessment)
2,1.0,3,901acc108f55a5a1,0006a69f,1.0,0.0,Mushroom Sorter (Assessment)
3,0.5,2,9501794defd84e4d,0006a69f,1.0,1.0,Mushroom Sorter (Assessment)
4,1.0,3,a9ef3ecb3d1acc6a,0006a69f,1.0,0.0,Bird Measurer (Assessment)
...,...,...,...,...,...,...,...
2,NaN,3,905d31d016c7a685,ffc73fb2,1.0,0.0,NaN
3,NaN,3,b46072c5e2b3d8c5,ffc73fb2,1.0,0.0,NaN
0,NaN,2,70336ec581799feb,ffe00ca8,1.0,1.0,NaN
1,NaN,1,c116d9e6f8cf85c3,ffe00ca8,1.0,3.0,NaN
